In [ ]:
import pandas as pd
import os
import tqdm
import pickle
from joblib import delayed, Parallel
FOLDER_ADDRESS = r'D:\Projects\stochastic-dominance'
os.chdir(FOLDER_ADDRESS)
from utils.prepare_data import clean_data
from dominance.dominance import StochasticDominance
price_data_address = os.path.join(FOLDER_ADDRESS,'data')

return_data = clean_data(price_data_address)

In [ ]:
sd = StochasticDominance(2, 'Passive', 80, 'SPY')
sd.get_dominance(return_data, pd.to_datetime('2018-08-20', utc=True), pd.to_datetime('2019-08-20', utc=True), ['AAPL', 'GOOG'])

In [ ]:
############ parameters you need to set
jstart_date = jdatetime.datetime.strptime('1394-01-01', '%Y-%m-%d').date()
jend_date = jdatetime.datetime.strptime('1394-02-01', '%Y-%m-%d').date()
#########################################


all_data = pd.read_pickle('Price_Data_Cleaned.pickle')
liquidity_filter = pd.read_pickle('liquidity_filter.pickle')
portfolio_calc_dates = []
portfolio_calc_dates.append(jstart_date)
end = jstart_date
while end < jend_date:
    end = add_month(end,  1)
    portfolio_calc_dates.append(end)


def par_func(date, lookback_for_dominance, order, active_passive):
    prev_month_date = subtract_month(date, lookback_for_dominance)
    tmp_liquidity = liquidity_filter[(liquidity_filter['Date'] >= d(prev_month_date)) & (liquidity_filter['Date'] <= d(date))]
    all_ncs = tmp_liquidity.groupby('NationalCode').last()
    all_ncs_sorted = all_ncs.sort_values(by='LiqScore', ascending=False)
    liquid_ncs = list(all_ncs_sorted[all_ncs_sorted['LiqScore'] == 1].index)
    tmp_data = all_data[(all_data['Date'] >= d(prev_month_date)) & (all_data['Date'] <= d(date))]
    tmp_data_liquid = tmp_data[tmp_data.index.isin(liquid_ncs)]
    del tmp_data, all_ncs
    segments = get_capital_segment(2, tmp_data_liquid)
    x = get_dominance(all_data, 
                      segments, 
                      d(prev_month_date), 
                      d(date), 
                      lookback_for_dominance, 
                      order, 
                      active_passive)
    return {date : x}


ddddd = par_func(portfolio_calc_dates[0], 3, 1, False)
# for l in [3]:
#     for o in [1, 2]:
#         for actv in [True, False]:
#             results = Parallel(n_jobs=4)(delayed(par_func)(date, l, o, actv) for date in tqdm.tqdm(portfolio_calc_dates))
#             with open(f'results_{l}_Month_{o}_Order_{actv}.pickle', 'wb') as handle:
#                 pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)